In [11]:
import polars as pl
import pandas as pd
import time
from pathlib import Path

In [12]:
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

data_path = project_root / "data" / "raw"
bronze_path = project_root / "data" / "bronze"

bronze_path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data path: {data_path}")
print(f"Bronze folder created: {bronze_path}")

Project root: c:\python-projects\F1-pipeline
Data path: c:\python-projects\F1-pipeline\data\raw
Bronze folder created: c:\python-projects\F1-pipeline\data\bronze


In [13]:
csv_files = list(data_path.glob("*.csv"))
print(f"Number of CSV files: {len(csv_files)}\n")
for file in csv_files:
    print(f"  - {file.name}")

Number of CSV files: 14

  - circuits.csv
  - constructors.csv
  - constructor_results.csv
  - constructor_standings.csv
  - drivers.csv
  - driver_standings.csv
  - lap_times.csv
  - pit_stops.csv
  - qualifying.csv
  - races.csv
  - results.csv
  - seasons.csv
  - sprint_results.csv
  - status.csv


In [14]:
print("\n" + "="*50)
print("PANDAS vs POLARS")
print("="*50)

start = time.time()
df_pandas = pd.read_csv(data_path / "circuits.csv")
pandas_time = time.time() - start
print(f"\n Pandas: {pandas_time:.4f} seconds")
print(f"  Shape: {df_pandas.shape}")
print(f" Memory: {df_pandas.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

start = time.time()
df_polars = pl.read_csv(data_path / "circuits.csv")
polars_time = time.time() - start
print(f"\n Polars: {polars_time:.4f} seconds")
print(f"  Shape: {df_polars.shape}")
print(f" Memory: {df_polars.estimated_size('mb'):.2f} MB")

print(f"\nPolars was {pandas_time / polars_time:.2f}x faster!")


PANDAS vs POLARS

 Pandas: 0.0049 seconds
  Shape: (77, 9)
 Memory: 0.03 MB

 Polars: 0.0010 seconds
  Shape: (77, 9)
 Memory: 0.01 MB

Polars was 4.89x faster!


In [15]:
print("\n" + "="*50)
print("DATASET STRUCTURE: circuits")
print("="*50)

print("\nSchema (data types):")
print(df_polars.schema)

print("\nFirst 5 rows:")
print(df_polars.head())

print("\nDescriptive statistics:")
print(df_polars.describe())


DATASET STRUCTURE: circuits

Schema (data types):
Schema([('circuitId', Int64), ('circuitRef', String), ('name', String), ('location', String), ('country', String), ('lat', Float64), ('lng', Float64), ('alt', Int64), ('url', String)])

First 5 rows:
shape: (5, 9)
┌───────────┬─────────────┬─────────────┬─────────────┬───┬──────────┬─────────┬─────┬─────────────┐
│ circuitId ┆ circuitRef  ┆ name        ┆ location    ┆ … ┆ lat      ┆ lng     ┆ alt ┆ url         │
│ ---       ┆ ---         ┆ ---         ┆ ---         ┆   ┆ ---      ┆ ---     ┆ --- ┆ ---         │
│ i64       ┆ str         ┆ str         ┆ str         ┆   ┆ f64      ┆ f64     ┆ i64 ┆ str         │
╞═══════════╪═════════════╪═════════════╪═════════════╪═══╪══════════╪═════════╪═════╪═════════════╡
│ 1         ┆ albert_park ┆ Albert Park ┆ Melbourne   ┆ … ┆ -37.8497 ┆ 144.968 ┆ 10  ┆ http://en.w │
│           ┆             ┆ Grand Prix  ┆             ┆   ┆          ┆         ┆     ┆ ikipedia.or │
│           ┆             ┆ 

In [16]:
null_counts = df_polars.null_count()
print(null_counts)

shape: (1, 9)
┌───────────┬────────────┬──────┬──────────┬───┬─────┬─────┬─────┬─────┐
│ circuitId ┆ circuitRef ┆ name ┆ location ┆ … ┆ lat ┆ lng ┆ alt ┆ url │
│ ---       ┆ ---        ┆ ---  ┆ ---      ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ u32       ┆ u32        ┆ u32  ┆ u32      ┆   ┆ u32 ┆ u32 ┆ u32 ┆ u32 │
╞═══════════╪════════════╪══════╪══════════╪═══╪═════╪═════╪═════╪═════╡
│ 0         ┆ 0          ┆ 0    ┆ 0        ┆ … ┆ 0   ┆ 0   ┆ 0   ┆ 0   │
└───────────┴────────────┴──────┴──────────┴───┴─────┴─────┴─────┴─────┘


## Save as Parquet

In [17]:
parquet_file = bronze_path / "circuits.parquet"
df_polars.write_parquet(parquet_file, compression="snappy")

# Compare sizes
csv_size = (data_path / "circuits.csv").stat().st_size / 1024
parquet_size = parquet_file.stat().st_size / 1024

print(f"\nOriginal CSV: {csv_size:.2f} KB")
print(f"Parquet: {parquet_size:.2f} KB")
print(f"File size reduction: {((csv_size - parquet_size) / csv_size * 100):.1f}%")


Original CSV: 9.87 KB
Parquet: 10.16 KB
File size reduction: -2.9%


In [18]:
start = time.time()
df_test = pl.read_parquet(parquet_file)
parquet_read_time = time.time() - start

print(f"Parquet read time: {parquet_read_time:.4f} seconds")
print(f"Read Parquet was {polars_time/parquet_read_time:.2f}x faster than CSV!")


Parquet read time: 0.0165 seconds
Read Parquet was 0.06x faster than CSV!


In [19]:
df_results = pl.read_csv(data_path / "results.csv", schema_overrides={"number": pl.Utf8, "points": pl.Float64})
print(f"Shape: {df_results.shape}")
print(f"Columns: {df_results.columns}")
print(f"Sample:")
print(df_results.head())

Shape: (26759, 18)
Columns: ['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId']
Sample:
shape: (5, 18)
┌──────────┬────────┬──────────┬───────────────┬───┬──────┬──────────────┬──────────────┬──────────┐
│ resultId ┆ raceId ┆ driverId ┆ constructorId ┆ … ┆ rank ┆ fastestLapTi ┆ fastestLapSp ┆ statusId │
│ ---      ┆ ---    ┆ ---      ┆ ---           ┆   ┆ ---  ┆ me           ┆ eed          ┆ ---      │
│ i64      ┆ i64    ┆ i64      ┆ i64           ┆   ┆ str  ┆ ---          ┆ ---          ┆ i64      │
│          ┆        ┆          ┆               ┆   ┆      ┆ str          ┆ str          ┆          │
╞══════════╪════════╪══════════╪═══════════════╪═══╪══════╪══════════════╪══════════════╪══════════╡
│ 1        ┆ 18     ┆ 1        ┆ 1             ┆ … ┆ 2    ┆ 1:27.452     ┆ 218.300      ┆ 1        │
│ 2        ┆ 1

In [ ]:
df_races = pl.read_csv(data_path / "races.csv")

print(f"Total races registered: {df_races.shape[0]}")
print(f"Total results: {df_results.shape[0]}")
print(f"Min year: {df_races['year'].min()}")
print(f"Max year: {df_races['year'].max()}")

print(f"\nRaces per year (last 5 years):")
races_by_year = (
    df_races
    .group_by("year")
    .agg(pl.len().alias("num_races"))
    .sort("year", descending=True)
    .head(5)
)
print(races_by_year)

Total races recorded: 1125
Total results: 26759
Minumum year: 1950
Maximum year: 2024

Races by year (last 5 years):
shape: (5, 2)
┌──────┬───────────┐
│ year ┆ num_races │
│ ---  ┆ ---       │
│ i64  ┆ u32       │
╞══════╪═══════════╡
│ 2024 ┆ 24        │
│ 2023 ┆ 22        │
│ 2022 ┆ 22        │
│ 2021 ┆ 22        │
│ 2020 ┆ 17        │
└──────┴───────────┘


In [42]:
# CSV
start = time.time()
df_laps_csv = pl.read_csv(data_path / "lap_times.csv")
csv_time = time.time() - start

# Save as Parquet
parquet_laps = bronze_path / "lap_times.parquet"
df_laps_csv.write_parquet(parquet_laps, compression="snappy")

# Read Parquet
start = time.time()
df_laps_parquet = pl.read_parquet(parquet_laps)
parquet_time = time.time() - start

# Compare
csv_size = (data_path / "lap_times.csv").stat().st_size / 1024**2
parquet_size = parquet_laps.stat().st_size / 1024**2

print(f"Dataset shape: {df_laps_csv.shape}")
print(f"\nCSV read time: {csv_time:.4f}s")
print(f"Parquet read time: {parquet_time:.4f}s")
print(f"Speedup: {csv_time/parquet_time:.2f}x faster")

print(f"\nCSV size: {csv_size:.2f} MB")
print(f"Parquet size: {parquet_size:.2f} MB")
print(f"Size reduction: {((csv_size - parquet_size) / csv_size * 100):.1f}%")


Dataset shape: (589081, 6)

CSV read time: 0.0219s
Parquet read time: 0.0272s
Speedup: 0.80x faster

CSV size: 16.81 MB
Parquet size: 3.92 MB
Size reduction: 76.7%


In [ ]:
# CSV
start = time.time()
#df_csv_selective = pl.read_csv(data_path / "lap_times.csv").select(["raceId", "milliseconds"])
df_csv_selective = pl.read_csv(data_path / "lap_times.csv", columns=["raceId", "milliseconds"])
csv_selective_time = time.time() - start

# Parquet
start = time.time()
df_parquet_selective = pl.read_parquet(parquet_laps, columns=["raceId", "milliseconds"])
parquet_selective_time = time.time() - start

print(f"CSV (read all, then select): {csv_selective_time:.4f}s")
print(f"Parquet (read only 2 cols): {parquet_selective_time:.4f}s")
print(f"Parquet is {csv_selective_time/parquet_selective_time:.2f}x faster!")

print(f"\nResult shape: {df_parquet_selective.shape}")

CSV (read all, then select): 0.0091s
Parquet (read only 2 cols): 0.0122s
Parquet is 0.74x faster!

Result shape: (589081, 2)
Parquet read only 33% of columns but got the same result!
